### actualizar clientes ###


In [1]:
from pyspark.sql import SparkSession

# Cerrar sesión actual
try:
    spark.stop()
    print("✅ Sesión anterior cerrada")
except:
    print("ℹ️ No había sesión previa")

# Reiniciar con configuración explícita del JAR
spark = (SparkSession.builder
    .appName("Validacion-update-clientes")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "1")
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "1g")
    .config("spark.jars", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.driver.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.executor.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .enableHiveSupport()
    .getOrCreate())

print("✅ Spark version:", spark.version)
print("✅ Master URL:", spark.sparkContext.master)
print("✅ Spark reiniciado con driver JDBC")

ℹ️ No había sesión previa


25/09/11 20:21:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark version: 2.4.5
✅ Master URL: spark://spark-master:7077
✅ Spark reiniciado con driver JDBC


In [ ]:
#from pyspark.sql import Row
# Crear DF con el watermark
#watermark_df = spark.createDataFrame([Row(tabla="update_clientes",last_watermark="2025-08-24 21:31:59.697")])

In [ ]:
#watermark_df.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/update_clientes_watermark")



#print("Dataset creado")

[Stage 2:=============================>                             (1 + 1) / 2]

Dataset creado


In [2]:
df_watermark= spark.read.parquet("hdfs://namenode:8020/bronze/update_clientes_watermark")

In [3]:
df_watermark.show()

[Stage 1:>                                                          (0 + 1) / 1]

+--------------------+---------------+
|      last_watermark|          tabla|
+--------------------+---------------+
|2025-08-24 21:31:...|update_clientes|
+--------------------+---------------+



In [4]:
marca_df = df_watermark.filter(df_watermark.tabla == "update_clientes") \
                       .select("last_watermark") \
                       .collect()[0][0]

print("🕒 Última marca de agua usada:", marca_df)

[Stage 3:=============================>                             (1 + 1) / 2]

🕒 Última marca de agua usada: 2025-08-24 21:31:59.697


In [6]:
query_clientes_incr = f"""
(SELECT *
 FROM dbo.Clientes
 WHERE UpdateTime > '{marca_df}') as clientes_incr
"""

df_updates = (spark.read.format("jdbc")
    .option("url", "jdbc:sqlserver://host.docker.internal:1433;databaseName=olva;encrypt=false")
    .option("dbtable", query_clientes_incr)
    .option("user", "etl_user")
    .option("password", "StrongPass123")
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
    .load())

df_updates.show(truncate=False)


+---------+--------------+----------------------------+----------+-----------------------+-----------------------+
|ClienteID|Nombre        |Email                       |Telefono  |CreateTime             |UpdateTime             |
+---------+--------------+----------------------------+----------+-----------------------+-----------------------+
|2        |María López   |mar23928.actualice@email.com|0987654321|2025-08-24 21:27:20.383|2025-09-11 17:22:59.786|
|14       |fabricio total|fabrico.update@email.com    |0987654321|2025-09-11 16:03:07.859|2025-09-11 20:32:42.701|
+---------+--------------+----------------------------+----------+-----------------------+-----------------------+



In [ ]:
#from pyspark.sql.functions import to_date, col

# Leer clientes sin partición
#df_clientes = spark.read.parquet("hdfs://namenode:8020/bronze/clientes")

# Crear columna de partición a partir de CreateTime
#df_clientes = df_clientes.withColumn("fecha", to_date(col("CreateTime")))

# Reescribir particionado por fecha (de creación)
#df_clientes.write.mode("overwrite") \
 #   .partitionBy("fecha") \
 #   .parquet("hdfs://namenode:8020/bronze/clientes_partitioned")

#print("✅ Datos reorganizados con particiones por CreateTime")


✅ Datos reorganizados con particiones por CreateTime


In [7]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

BRONZE_PATH = "hdfs://namenode:8020/bronze/clientes_partitioned"
WATERMARK_PATH = "hdfs://namenode:8020/bronze/update_clientes_watermark"


In [8]:
# Agrega la columna de partición (fecha) a partir de CreateTime
df_updates = df_updates.withColumn("fecha", F.to_date(F.col("CreateTime")))

# Opcional: cache si el lote es mediano/grande
df_updates.cache()
df_updates.count()  # fuerza materialización


2

In [9]:
particiones_afectadas = [r["fecha"] for r in df_updates.select("fecha").distinct().collect()]
print("📌 Particiones afectadas:", particiones_afectadas)

if not particiones_afectadas:
    print("✅ No hay filas nuevas/actualizadas. Nada que sobrescribir.")


[Stage 9:====================================================>  (190 + 2) / 200]

📌 Particiones afectadas: [datetime.date(2025, 9, 11), datetime.date(2025, 8, 24)]


In [10]:
df_existente = (spark.read.parquet(BRONZE_PATH)
                .filter(F.col("fecha").isin(particiones_afectadas)))


In [12]:
# Reordenar columnas de df_updates al mismo orden de df_existente
df_updates = df_updates.select(df_existente.columns)

# Unión segura
df_union = df_existente.unionByName(df_updates)

# Última versión por (fecha, ClienteID) usando UpdateTime
w = Window.partitionBy("fecha", "ClienteID").orderBy(F.col("UpdateTime").desc())

df_final = (df_union
            .withColumn("rn", F.row_number().over(w))
            .filter(F.col("rn") == 1)
            .drop("rn"))

df_final.count(), df_final.select("fecha").distinct().show()



+----------+
|     fecha|
+----------+
|2025-09-11|
|2025-08-24|
+----------+



(8, None)

In [13]:
(df_final.write
    .mode("overwrite")
    .option("partitionOverwriteMode", "dynamic")
    .partitionBy("fecha")
    .parquet(BRONZE_PATH))

print("✅ Particiones sobrescritas dinámicamente en Bronze")


[Stage 30:================================================>     (181 + 2) / 200]

✅ Particiones sobrescritas dinámicamente en Bronze


In [14]:
for f in ["2025-08-24", "2025-09-11"]:
    print(f"🔎 Vista partición fecha={f}:")
    (spark.read.parquet(BRONZE_PATH)
          .filter(F.col("fecha") == f)
          .orderBy("ClienteID")
          .show(truncate=False))


🔎 Vista partición fecha=2025-08-24:
+---------+-----------+----------------------------+----------+-----------------------+-----------------------+----------+
|ClienteID|Nombre     |Email                       |Telefono  |CreateTime             |UpdateTime             |fecha     |
+---------+-----------+----------------------------+----------+-----------------------+-----------------------+----------+
|1        |Juan Pérez |juan.perez@email.com        |089000100 |2025-08-24 21:27:20.383|2025-08-24 21:31:48.536|2025-08-24|
|2        |María López|mar23928.actualice@email.com|0987654321|2025-08-24 21:27:20.383|2025-09-11 17:22:59.786|2025-08-24|
|3        |Carlos Ruiz|carlos.ruiz@email.com       |0971122334|2025-08-24 21:27:20.383|null                   |2025-08-24|
|4        |Ana Torres |ana.torres@email.com        |0965544332|2025-08-24 21:27:20.383|null                   |2025-08-24|
|5        |Pedro Gómez|pedro.gomez@email.com       |0956677889|2025-08-24 21:27:20.383|null            

In [15]:
df_bronze = spark.read.parquet(BRONZE_PATH)
df_bronze.show(30)

+---------+--------------+--------------------+----------+--------------------+--------------------+----------+
|ClienteID|        Nombre|               Email|  Telefono|          CreateTime|          UpdateTime|     fecha|
+---------+--------------+--------------------+----------+--------------------+--------------------+----------+
|        2|   María López|mar23928.actualic...|0987654321|2025-08-24 21:27:...|2025-09-11 17:22:...|2025-08-24|
|       14|fabricio total|fabrico.update@em...|0987654321|2025-09-11 16:03:...|2025-09-11 20:32:...|2025-09-11|
|        1|    Juan Pérez|juan.perez@email.com| 089000100|2025-08-24 21:27:...|2025-08-24 21:31:...|2025-08-24|
|       15|   Carlos Ruiz|carlos.toala@emai...|0971122334|2025-09-11 16:03:...|                null|2025-09-11|
|        5|   Pedro Gómez|pedro.gomez@email...|0956677889|2025-08-24 21:27:...|                null|2025-08-24|
|       13|  leodan toala|juan.toala@email.com|0991234567|2025-09-11 16:02:...|                null|2025

In [16]:
from pyspark.sql.functions import max as spark_max

# Calcular el nuevo máximo de UpdateTime en los updates procesados
nueva_marca = df_updates.agg(spark_max("UpdateTime")).collect()[0][0]

print("🕒 Nueva marca de agua:", nueva_marca)


🕒 Nueva marca de agua: 2025-09-11 20:32:42.701000


In [20]:
from pyspark.sql import Row

df_nueva_marca = spark.createDataFrame([
    Row(last_watermark=nueva_marca, tabla="update_clientes")
])

In [21]:
df_nueva_marca.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/update_clientes_watermark")


In [22]:
df = spark.read.parquet("hdfs://namenode:8020/bronze/update_clientes_watermark")

In [26]:
df.show(truncate=False)

+-----------------------+---------------+
|last_watermark         |tabla          |
+-----------------------+---------------+
|2025-09-11 20:32:42.701|update_clientes|
+-----------------------+---------------+



In [29]:
spark.catalog.clearCache()
print("✅ Cache de Spark liberado")


✅ Cache de Spark liberado


In [19]:
spark.stop()